In [199]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [200]:
# Load datasets
df1 = pd.read_csv('/home/wahba/Documents/cicid/cicids2017/original/csv/TrafficLabelling /Wednesday-workingHours.pcap_ISCX.csv')
df2 = pd.read_csv('/home/wahba/Documents/cicid/CICFlowMeter/data/daily/2025-11-21_Flow.csv')
df3 = pd.read_csv('/home/wahba/Documents/nids3/tests/flow/flow_tuesday_flow_generation4.csv')

In [202]:
# Cleaning column names
col_names = {col: col.strip() for col in df1.columns}
df1.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df2.columns}
df2.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df3.columns}
df3.rename(columns=col_names, inplace=True)

del col_names

In [203]:
df1['Label'].value_counts()

Label
BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name: count, dtype: int64

In [206]:
# Filter out only DoS slowhttptest attacks
df1_dos = df1[df1['Label'] == 'DoS Slowhttptest']

In [ ]:
df1_dos['flow_key'] = (
    df1_dos['Source IP'].astype(str) + ':' +
    df1_dos['Source Port'].astype(str) + '-' +
    df1_dos['Destination IP'].astype(str) + ':' +
    df1_dos['Destination Port'].astype(str) + '-' +
    df1_dos['Protocol'].astype(str)
    )
df2['flow_key'] = (
    df2['Src IP'].astype(str) + ':' +
    df2['Src Port'].astype(str) + '-' +
    df2['Dst IP'].astype(str) + ':' +
    df2['Dst Port'].astype(str) + '-' +
    df2['Protocol'].astype(str)
    )
df3['Protocol'] = df3['Protocol'].replace({'TCP': 6, 'UDP': 17, 'ICMP': 1}).astype(int)
df3['flow_key'] = (
    df3['Source IP'].astype(str) + ':' +
    df3['Source Port'].astype(str) + '-' +
    df3['Destination IP'].astype(str) + ':' +
    df3['Destination Port'].astype(str) + '-' +
    df3['Protocol'].astype(str)
    )

In [211]:
# Get unique flow keys for slowhttptest from df1

slowhttptest_flow_key = df1_dos['flow_key'].unique()

In [ ]:
# Filter df2 and df3 for flows matching slowhttptest flow keys
df2_dos = df2[df2['flow_key'].isin(slowhttptest_flow_key)]
df3_dos = df3[df3['flow_key'].isin(slowhttptest_flow_key)]

In [213]:
print(df2_dos.shape)
print(df3_dos.shape)

(659, 85)
(376, 58)


In [171]:
len(set(df2['flow_key']).intersection(set(df3['flow_key'])))

934

In [174]:
len(set(df2['flow_key']).difference(set(df3['flow_key'])))

2078

In [175]:
len(df3)

1504

In [177]:
df3_match_df2 = df3[df3['flow_key'].isin(df2['flow_key'])]
df3_match_df2.shape

(1191, 58)

In [179]:
df3_match_df2['flow_key'].value_counts()

flow_key
192.168.10.8:51422-72.21.91.29:80-6         4
192.168.10.15:51282-172.217.10.130:443-6    4
192.168.10.14:51229-72.21.91.29:80-6        4
192.168.10.50:54722-192.168.10.3:389-6      4
192.168.10.50:53708-192.168.10.3:3268-6     4
                                           ..
192.168.10.9:16149-35.160.9.151:443-6       1
192.168.10.25:50569-89.36.26.156:443-6      1
172.16.0.1:35048-192.168.10.50:80-6         1
172.16.0.1:33860-192.168.10.50:80-6         1
172.16.0.1:35028-192.168.10.50:80-6         1
Name: count, Length: 934, dtype: int64

In [ ]:
# Based on the above, from 3852 flow in df2, only 350 are DoS Slowhttptest

In [ ]:
# finding duplication of flows indicated as attacks in df2
slowhttptest_flows = df2[df2['Flow ID'].isin(set(df1_dos['Flow ID']))]
print("number of flows:", len(slowhttptest_flows['Flow ID']))
print("number of unique flows:", slowhttptest_flows['Flow ID'].nunique())

number of flows: 659
number of unique flows: 350


# Test model prediction on flow generated by CICFLOWMETER and from CICIDS2017

In [180]:
import joblib

rf = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
xgb = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
knn = joblib.load("/home/wahba/Documents/models_anacletu/knn_model.joblib")
robust_scaler = joblib.load("/home/wahba/Documents/models_anacletu/robust_scaler.joblib")

/home/wahba/miniconda3/envs/ml2/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/wahba/miniconda3/envs/ml2/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Standardizing dataframes to feed to ML model

In [ ]:
# Standardizing column names
'''
final_cols_df1 = {
    'Source IP': 'Source IP',
    'Destination IP': 'Destination IP',
    'Protocol': 'Protocol',
    'Source Port': 'Source Port',
    'Destination Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Total Fwd Packets',
    'Total Length of Fwd Packets': 'Total Length of Fwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Min Packet Length': 'Min Packet Length',
    'Max Packet Length': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'Average Packet Size': 'Average Packet Size',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Init_Win_bytes_forward': 'Init_Win_bytes_forward',
    'Init_Win_bytes_backward': 'Init_Win_bytes_backward',
    'act_data_pkt_fwd': 'act_data_pkt_fwd',
    'min_seg_size_forward': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

final_cols_df2 = {
    'Src IP': 'Source IP',
    'Dst IP': 'Destination IP',
    'Protocol': 'Protocol',
    'Src Port': 'Source Port',
    'Dst Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packet': 'Total Fwd Packets',
    'Total Length of Fwd Packet': 'Total Length of Fwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Packet Length Min': 'Min Packet Length',
    'Packet Length Max': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'Average Packet Size': 'Average Packet Size',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'FWD Init Win Bytes': 'Init_Win_bytes_forward',
    'Bwd Init Win Bytes': 'Init_Win_bytes_backward',
    'Fwd Act Data Pkts': 'act_data_pkt_fwd',
    'Fwd Seg Size Min': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}
'''

final_columns_df1 = {
    'Destination Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Total Fwd Packets',
    'Total Length of Fwd Packets': 'Total Length of Fwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Min Packet Length': 'Min Packet Length',
    'Max Packet Length': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'Average Packet Size': 'Average Packet Size',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Init_Win_bytes_forward': 'Init_Win_bytes_forward',
    'Init_Win_bytes_backward': 'Init_Win_bytes_backward',
    'act_data_pkt_fwd': 'act_data_pkt_fwd',
    'min_seg_size_forward': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

final_columns_df2 = {
    'Dst Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packet': 'Total Fwd Packets',
    'Total Length of Fwd Packet': 'Total Length of Fwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Packet Length Min': 'Min Packet Length',
    'Packet Length Max': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'Average Packet Size': 'Average Packet Size',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'FWD Init Win Bytes': 'Init_Win_bytes_forward',
    'Bwd Init Win Bytes': 'Init_Win_bytes_backward',
    'Fwd Act Data Pkts': 'act_data_pkt_fwd',
    'Fwd Seg Size Min': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

In [181]:
df1_dos_standardized = df1_dos[list(final_columns_df1.keys())].rename(columns=final_columns_df1)
df2_standardized = df2[list(final_columns_df2.keys())].rename(columns=final_columns_df2)
df3_match_df2_standardized = df3_match_df2[list(final_columns_df1.keys())].rename(columns=final_columns_df1)
print("expected number of features for the model:", len(final_columns_df1))

expected number of features for the model: 52


In [182]:
print("df1_dos_standardized shape:", df1_dos_standardized.shape)
print("df2_standardized shape:", df2_standardized.shape)
print("df3_match_df2_standardized shape:", df3_match_df2_standardized.shape)
print("Expected number of features for the model:", len(final_columns_df1))

df1_dos_standardized shape: (5499, 52)
df2_standardized shape: (3852, 52)
df3_match_df2_standardized shape: (1191, 52)
Expected number of features for the model: 52


# Model prediction

In [183]:
import joblib

rf = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
xgb = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
knn = joblib.load("/home/wahba/Documents/models_anacletu/knn_model.joblib")
robust_scaler = joblib.load("/home/wahba/Documents/models_anacletu/robust_scaler.joblib")

/home/wahba/miniconda3/envs/ml2/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/wahba/miniconda3/envs/ml2/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [184]:
print(rf.classes_)
print(xgb.classes_)
print(knn.classes_)

[0 1 2 3 4 5 6]
[0 1 2 3 4 5 6]
['Bots' 'Brute Force' 'DDoS' 'DoS' 'Normal Traffic' 'Port Scanning'
 'Web Attacks']


In [193]:
rf_predictions_df1 = rf.predict_proba(df1_dos_standardized)
xgb_predictions_df1 = xgb.predict_proba(df1_dos_standardized)
knn_predictions_df1 = knn.predict_proba(robust_scaler.transform(df1_dos_standardized))

rf_predictions_df2 = rf.predict_proba(df2_standardized)
xgb_predictions_df2 = xgb.predict_proba(df2_standardized)
knn_predictions_df2 = knn.predict_proba(robust_scaler.transform(df2_standardized))

rf_predictions_df3 = rf.predict_proba(df3_match_df2_standardized)
xgb_predictions_df3 = xgb.predict_proba(df3_match_df2_standardized)
knn_predictions_df3 = knn.predict_proba(robust_scaler.transform(df3_match_df2_standardized))

In [195]:
import pandas as pd

# Create a summary dataframe
results = []

datasets = ['df1', 'df2', 'df3']
models = ['rf', 'xgb', 'knn']
predictions_dict = {
    'df1': {'rf': rf_predictions_df1, 'xgb': xgb_predictions_df1, 'knn': knn_predictions_df1},
    'df2': {'rf': rf_predictions_df2, 'xgb': xgb_predictions_df2, 'knn': knn_predictions_df2},
    'df3': {'rf': rf_predictions_df3, 'xgb': xgb_predictions_df3, 'knn': knn_predictions_df3}
}

for dataset in datasets:
    for model in models:
        preds = predictions_dict[dataset][model]
        value_counts = pd.Series(preds.argmax(axis=1) if len(preds.shape) > 1 else preds).value_counts().sort_index()
        
        for class_label, count in value_counts.items():
            results.append({
                'Dataset': dataset,
                'Model': model.upper(),
                'Class': class_label,
                'Count': count,
                'Percentage': f"{(count/len(preds)*100):.1f}%"
            })

summary_df = pd.DataFrame(results)
print(summary_df.pivot_table(index=['Dataset', 'Model'], columns='Class', values=['Count', 'Percentage'], aggfunc='first'))

                Count                                          Percentage  \
Class               0       1     2       3       4     5    6          0   
Dataset Model                                                               
df1     KNN       NaN     NaN   1.0  5308.0   189.0   NaN  1.0        NaN   
        RF        4.0  5493.0   NaN     1.0     NaN   1.0  NaN       0.1%   
        XGB       4.0  5493.0   NaN     1.0     NaN   1.0  NaN       0.1%   
df2     KNN       NaN     1.0  41.0   434.0  3359.0  17.0  NaN        NaN   
        RF     3193.0   659.0   NaN     NaN     NaN   NaN  NaN      82.9%   
        XGB    3193.0   659.0   NaN     NaN     NaN   NaN  NaN      82.9%   
df3     KNN       NaN     NaN   8.0   281.0   902.0   NaN  NaN        NaN   
        RF      989.0   202.0   NaN     NaN     NaN   NaN  NaN      83.0%   
        XGB     989.0   202.0   NaN     NaN     NaN   NaN  NaN      83.0%   

                                                      
Class              1

In [196]:
def display_predictions_summary(predictions_dict, dataset_names=None, model_names=None):
    if dataset_names is None:
        dataset_names = list(predictions_dict.keys())
    if model_names is None:
        model_names = list(predictions_dict[dataset_names[0]].keys())
    
    for dataset in dataset_names:
        print(f"\n{'='*50}")
        print(f"DATASET: {dataset.upper()}")
        print(f"{'='*50}")
        
        for model in model_names:
            preds = predictions_dict[dataset][model]
            # Handle both binary and multiclass probability arrays
            if len(preds.shape) > 1:
                class_predictions = preds.argmax(axis=1)
            else:
                class_predictions = (preds > 0.5).astype(int)
            
            value_counts = pd.Series(class_predictions).value_counts().sort_index()
            
            print(f"\n{model.upper()} Predictions:")
            for class_label, count in value_counts.items():
                percentage = (count / len(preds)) * 100
                print(f"  Class {class_label}: {count} ({percentage:.1f}%)")

# Usage
predictions_dict = {
    'df1': {'rf': rf_predictions_df1, 'xgb': xgb_predictions_df1, 'knn': knn_predictions_df1},
    'df2': {'rf': rf_predictions_df2, 'xgb': xgb_predictions_df2, 'knn': knn_predictions_df2},
    'df3': {'rf': rf_predictions_df3, 'xgb': xgb_predictions_df3, 'knn': knn_predictions_df3}
}

display_predictions_summary(predictions_dict)


DATASET: DF1

RF Predictions:
  Class 0: 4 (0.1%)
  Class 1: 5493 (99.9%)
  Class 3: 1 (0.0%)
  Class 5: 1 (0.0%)

XGB Predictions:
  Class 0: 4 (0.1%)
  Class 1: 5493 (99.9%)
  Class 3: 1 (0.0%)
  Class 5: 1 (0.0%)

KNN Predictions:
  Class 2: 1 (0.0%)
  Class 3: 5308 (96.5%)
  Class 4: 189 (3.4%)
  Class 6: 1 (0.0%)

DATASET: DF2

RF Predictions:
  Class 0: 3193 (82.9%)
  Class 1: 659 (17.1%)

XGB Predictions:
  Class 0: 3193 (82.9%)
  Class 1: 659 (17.1%)

KNN Predictions:
  Class 1: 1 (0.0%)
  Class 2: 41 (1.1%)
  Class 3: 434 (11.3%)
  Class 4: 3359 (87.2%)
  Class 5: 17 (0.4%)

DATASET: DF3

RF Predictions:
  Class 0: 989 (83.0%)
  Class 1: 202 (17.0%)

XGB Predictions:
  Class 0: 989 (83.0%)
  Class 1: 202 (17.0%)

KNN Predictions:
  Class 2: 8 (0.7%)
  Class 3: 281 (23.6%)
  Class 4: 902 (75.7%)


In [194]:
rf_predictions_df1_pd = pd.Series(rf_predictions_df1)
xgb_predictions_df1_pd = pd.Series(xgb_predictions_df1)
knn_predictions_df1_pd = pd.Series(knn_predictions_df1)

rf_predictions_df2_pd = pd.Series(rf_predictions_df2)
xgb_predictions_df2_pd = pd.Series(xgb_predictions_df2)
knn_predictions_df2_pd = pd.Series(knn_predictions_df2)

rf_predictions_df3_pd = pd.Series(rf_predictions_df3)
xgb_predictions_df3_pd = pd.Series(xgb_predictions_df3)
knn_predictions_df3_pd = pd.Series(knn_predictions_df3)


ValueError: Data must be 1-dimensional, got ndarray of shape (5499, 7) instead

In [192]:
print("rf preditions df1\n",rf_predictions_df1_pd.value_counts(), "\n")
print("xgb preditions df1\n",xgb_predictions_df1_pd.value_counts(), "\n")
print("knn preditions df1\n",knn_predictions_df1_pd.value_counts(), "\n")
print("rf preditions df2\n",rf_predictions_df2_pd.value_counts(), "\n")
print("xgb preditions df2\n",xgb_predictions_df2_pd.value_counts(), "\n")
print("knn preditions df2\n",knn_predictions_df2_pd.value_counts(), "\n")
print("rf preditions df3\n",rf_predictions_df3_pd.value_counts(), "\n")
print("xgb preditions df3\n",xgb_predictions_df3_pd.value_counts(), "\n")
print("knn preditions df3\n",knn_predictions_df3_pd.value_counts(), "\n")

rf preditions df1
 1    5493
0       4
3       1
5       1
Name: count, dtype: int64 

xgb preditions df1
 1    5493
0       4
3       1
5       1
Name: count, dtype: int64 

knn preditions df1
 DoS               5308
Normal Traffic     189
Web Attacks          1
DDoS                 1
Name: count, dtype: int64 

rf preditions df2
 0    3193
1     659
Name: count, dtype: int64 

xgb preditions df2
 0    3193
1     659
Name: count, dtype: int64 

knn preditions df2
 Normal Traffic    3359
DoS                434
DDoS                41
Port Scanning       17
Brute Force          1
Name: count, dtype: int64 

rf preditions df3
 0    989
1    202
Name: count, dtype: int64 

xgb preditions df3
 0    989
1    202
Name: count, dtype: int64 

knn preditions df3
 Normal Traffic    902
DoS               281
DDoS                8
Name: count, dtype: int64 



In [197]:
def display_average_probabilities(predictions_dict):
    """
    Display average probability of the predicted class for each model and dataset
    """
    results = []
    
    for dataset_name, models in predictions_dict.items():
        print(f"\n{'='*60}")
        print(f"DATASET: {dataset_name.upper()}")
        print(f"{'='*60}")
        
        for model_name, predictions in models.items():
            # Get the predicted class and its probability
            if len(predictions.shape) > 1:  # Multiclass probability array
                predicted_classes = predictions.argmax(axis=1)
                predicted_probs = predictions[np.arange(len(predictions)), predicted_classes]
            else:  # Binary classification probabilities
                predicted_classes = (predictions > 0.5).astype(int)
                predicted_probs = np.where(predicted_classes == 1, predictions, 1 - predictions)
            
            avg_prob = predicted_probs.mean()
            std_prob = predicted_probs.std()
            confidence_ratio = (predicted_probs > 0.5).mean()  # % of predictions with >50% confidence
            
            results.append({
                'Dataset': dataset_name,
                'Model': model_name.upper(),
                'Avg_Probability': avg_prob,
                'Std_Probability': std_prob,
                'High_Confidence_Ratio': confidence_ratio,
                'Samples': len(predictions)
            })
            
            print(f"{model_name.upper():<6}: Avg Probability = {avg_prob:.3f} ± {std_prob:.3f} "
                  f"(High Confidence: {confidence_ratio:.1%})")
    
    return pd.DataFrame(results)

# Usage
predictions_dict = {
    'df1': {'rf': rf_predictions_df1, 'xgb': xgb_predictions_df1, 'knn': knn_predictions_df1},
    'df2': {'rf': rf_predictions_df2, 'xgb': xgb_predictions_df2, 'knn': knn_predictions_df2},
    'df3': {'rf': rf_predictions_df3, 'xgb': xgb_predictions_df3, 'knn': knn_predictions_df3}
}

summary_df = display_average_probabilities(predictions_dict)


DATASET: DF1
RF    : Avg Probability = 0.997 ± 0.024 (High Confidence: 99.9%)
XGB   : Avg Probability = 0.997 ± 0.024 (High Confidence: 99.9%)
KNN   : Avg Probability = 0.995 ± 0.038 (High Confidence: 100.0%)

DATASET: DF2
RF    : Avg Probability = 0.999 ± 0.011 (High Confidence: 100.0%)
XGB   : Avg Probability = 0.999 ± 0.011 (High Confidence: 100.0%)
KNN   : Avg Probability = 0.987 ± 0.068 (High Confidence: 100.0%)

DATASET: DF3
RF    : Avg Probability = 0.983 ± 0.051 (High Confidence: 100.0%)
XGB   : Avg Probability = 0.983 ± 0.051 (High Confidence: 100.0%)
KNN   : Avg Probability = 0.985 ± 0.069 (High Confidence: 100.0%)
